![spaCy](assets/spacy.jpg)

## Setup

### Imports

In [3]:
%matplotlib inline
import sys
import tarfile
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import spacy
from spacy import displacy
import textacy

from IPython.display import SVG, display

In [4]:
pd.set_option('float_format', '{:,.2f}'.format)

### SpaCy Language Model Installation

In addition to the `spaCy` library, we need [language models](https://spacy.io/usage/models).

#### English

In [5]:
%%bash
python -m spacy download en_core_web_sm

# more comprehensive models:
# {sys.executable} -m spacy download en_core_web_md
# {sys.executable} -m spacy download en_core_web_lg


    Linking successful
    /home/stefan/.pyenv/versions/3.6.4/envs/at-3.6/lib/python3.6/site-packages/en_core_web_sm
    -->
    /home/stefan/.pyenv/versions/3.6.4/envs/at-3.6/lib/python3.6/site-packages/spacy/data/en_core_web_sm

    You can now load the model via spacy.load('en_core_web_sm')



#### Spanish

[Spanish language models](https://spacy.io/models/es#es_core_news_sm) trained on [AnCora Corpus](http://clic.ub.edu/corpus/) and [WikiNER](http://schwa.org/projects/resources/wiki/Wikiner)

In [6]:
%%bash
python -m spacy download es_core_news_sm

# more comprehensive model:
# {sys.executable} -m spacy download es_core_news_md


    Linking successful
    /home/stefan/.pyenv/versions/3.6.4/envs/at-3.6/lib/python3.6/site-packages/es_core_news_sm
    -->
    /home/stefan/.pyenv/versions/3.6.4/envs/at-3.6/lib/python3.6/site-packages/spacy/data/es_core_news_sm

    You can now load the model via spacy.load('es_core_news_sm')



In [7]:
%%bash
python -m spacy link en_core_web_sm en;
python -m spacy link es_core_news_sm es;


    Link 'en' already exists
    To overwrite an existing link, use the --force flag.


    Link 'es' already exists
    To overwrite an existing link, use the --force flag.



#### Validate Installation

In [8]:
# validate installation
!{sys.executable} -m spacy validate


    Installed models (spaCy v2.0.16)
    /home/stefan/.pyenv/versions/at-3.6/lib/python3.6/site-packages/spacy

    TYPE        NAME                  MODEL                 VERSION                                   
    package     es-core-news-sm       es_core_news_sm       2.0.0    ✔      
    package     en-core-web-sm        en_core_web_sm        2.0.0    ✔      
    link        en_core_web_sm        en_core_web_sm        2.0.0    ✔      
    link        en                    en_core_web_sm        2.0.0    ✔      
    link        es                    es_core_news_sm       2.0.0    ✔      
    link        es_core_news_sm       es_core_news_sm       2.0.0    ✔      


## Get Data

Download and unzip into `data` folder

- [BBC Articles](http://mlg.ucd.ie/datasets/bbc.html), use raw text files
- [TED2013](http://opus.nlpl.eu/TED2013.php), a parallel corpus of TED talk subtitles in 15 langugages

## SpaCy Pipeline & Architecture

### The Processing Pipeline

When you call a spaCy model on a text, spaCy 

1) tokenizes the text to produce a `Doc` object. 

2) passes the `Doc` object through the processing pipeline that may be customized, and for the default models consists of
- a tagger, 
- a parser and 
- an entity recognizer. 

Each pipeline component returns the processed Doc, which is then passed on to the next component.

![Architecture](assets/pipeline.svg)

### Key Data Structures

The central data structures in spaCy are the **Doc** and the **Vocab**. Text annotations are also designed to allow a single source of truth:

- The **`Doc`** object owns the sequence of tokens and all their annotations. `Span` and `Token` are views that point into it. It is constructed by the `Tokenizer`, and then modified in place by the components of the pipeline. 
- The **`Vocab`** object owns a set of look-up tables that make common information available across documents. 
- The **`Language`** object coordinates these components. It takes raw text and sends it through the pipeline, returning an annotated document. It also orchestrates training and serialization.

![Architecture](assets/spaCy-architecture.svg)

## SpaCy in Action

### Create & Explore the Language Object

In [9]:
nlp = spacy.load('en') 

In [10]:
type(nlp)

spacy.lang.en.English

In [11]:
nlp.lang

'en'

In [12]:
spacy.info('en_core_web_sm')


    Info about model en_core_web_sm

    lang               en             
    pipeline           ['tagger', 'parser', 'ner']
    accuracy           {'token_acc': 99.8698372794, 'ents_p': 84.9664503965, 'ents_r': 85.6312524451, 'uas': 91.7237657538, 'tags_acc': 97.0403350292, 'ents_f': 85.2975560875, 'las': 89.800872413}
    name               core_web_sm    
    license            CC BY-SA 3.0   
    author             Explosion AI   
    url                https://explosion.ai
    vectors            {'keys': 0, 'width': 0, 'vectors': 0}
    sources            ['OntoNotes 5', 'Common Crawl']
    version            2.0.0          
    spacy_version      >=2.0.0a18     
    parent_package     spacy          
    speed              {'gpu': None, 'nwords': 291344, 'cpu': 5122.3040471407}
    email              contact@explosion.ai
    description        English multi-task CNN trained on OntoNotes, with GloVe vectors trained on Common Crawl. Assigns word vectors, context-specific token v

{'lang': 'en',
 'pipeline': ['tagger', 'parser', 'ner'],
 'accuracy': {'token_acc': 99.8698372794,
  'ents_p': 84.9664503965,
  'ents_r': 85.6312524451,
  'uas': 91.7237657538,
  'tags_acc': 97.0403350292,
  'ents_f': 85.2975560875,
  'las': 89.800872413},
 'name': 'core_web_sm',
 'license': 'CC BY-SA 3.0',
 'author': 'Explosion AI',
 'url': 'https://explosion.ai',
 'vectors': {'keys': 0, 'width': 0, 'vectors': 0},
 'sources': ['OntoNotes 5', 'Common Crawl'],
 'version': '2.0.0',
 'spacy_version': '>=2.0.0a18',
 'parent_package': 'spacy',
 'speed': {'gpu': None, 'nwords': 291344, 'cpu': 5122.3040471407},
 'email': 'contact@explosion.ai',
 'description': 'English multi-task CNN trained on OntoNotes, with GloVe vectors trained on Common Crawl. Assigns word vectors, context-specific token vectors, POS tags, dependency parse and named entities.',
 'link': '/home/stefan/.pyenv/versions/at-3.6/lib/python3.6/site-packages/en_core_web_sm',
 'source': '/home/stefan/.pyenv/versions/3.6.4/envs/at

In [13]:
def get_attributes(f):
    print([a for a in dir(f) if not a.startswith('_')], end=' ')

In [14]:
get_attributes(nlp)

['Defaults', 'add_pipe', 'begin_training', 'create_pipe', 'disable_pipes', 'entity', 'evaluate', 'factories', 'from_bytes', 'from_disk', 'get_pipe', 'has_pipe', 'lang', 'make_doc', 'matcher', 'max_length', 'meta', 'parser', 'path', 'pipe', 'pipe_names', 'pipeline', 'preprocess_gold', 'remove_pipe', 'rename_pipe', 'replace_pipe', 'tagger', 'tensorizer', 'to_bytes', 'to_disk', 'tokenizer', 'update', 'use_params', 'vocab'] 

### Explore the Pipeline

In [15]:
sample_text = 'Apple is looking at buying U.K. startup for $1 billion'
doc = nlp(sample_text)

In [16]:
get_attributes(doc)

['cats', 'char_span', 'count_by', 'doc', 'ents', 'extend_tensor', 'from_array', 'from_bytes', 'from_disk', 'get_extension', 'get_lca_matrix', 'has_extension', 'has_vector', 'is_parsed', 'is_sentenced', 'is_tagged', 'mem', 'merge', 'noun_chunks', 'noun_chunks_iterator', 'print_tree', 'remove_extension', 'retokenize', 'sentiment', 'sents', 'set_extension', 'similarity', 'tensor', 'text', 'text_with_ws', 'to_array', 'to_bytes', 'to_disk', 'user_data', 'user_hooks', 'user_span_hooks', 'user_token_hooks', 'vector', 'vector_norm', 'vocab'] 

In [17]:
doc.is_parsed

True

In [18]:
doc.is_sentenced

True

In [19]:
doc.is_tagged

True

In [20]:
doc.text

'Apple is looking at buying U.K. startup for $1 billion'

In [21]:
get_attributes(doc.vocab)

['add_flag', 'cfg', 'data_dir', 'from_bytes', 'from_disk', 'get_vector', 'has_vector', 'lang', 'length', 'lex_attr_getters', 'lexemes_from_bytes', 'lexemes_to_bytes', 'morphology', 'prune_vectors', 'reset_vectors', 'set_vector', 'strings', 'to_bytes', 'to_disk', 'vectors', 'vectors_length'] 

In [22]:
doc.vocab.length

57852

#### Explore `Token` annotations

In [23]:
pd.Series([token.text for token in doc])

0       Apple
1          is
2     looking
3          at
4      buying
5        U.K.
6     startup
7         for
8           $
9           1
10    billion
dtype: object

In [26]:
pd.DataFrame([[t.text, t.lemma_, t.pos_, t.tag_, t.dep_, t.shape_, t.is_alpha, t.is_stop]
              for t in doc],
             columns=['text', 'lemma', 'pos', 'tag', 'dep', 'shape', 'is_alpha', 'is_stop'])

,text,lemma,pos,tag,dep,shape,is_alpha,is_stop
0,Apple,apple,PROPN,NNP,nsubj,Xxxxx,True,False
1,is,be,VERB,VBZ,aux,xx,True,True
2,looking,look,VERB,VBG,ROOT,xxxx,True,False
3,at,at,ADP,IN,prep,xx,True,True
4,buying,buy,VERB,VBG,pcomp,xxxx,True,False
5,U.K.,u.k.,PROPN,NNP,compound,X.X.,False,False
6,startup,startup,NOUN,NN,dobj,xxxx,True,False
7,for,for,ADP,IN,prep,xxx,True,True
8,$,$,SYM,$,quantmod,$,False,False
9,1,1,NUM,CD,compound,d,False,False


#### Visualize POS Dependencies

In [27]:
options = {'compact': True, 'bg': '#09a3d5',
           'color': 'white', 'font': 'Source Sans Pro', 'notebook': True}
displacy.serve(doc, style='dep', options=options)


    Serving on port 5000...
    Using the 'dep' visualizer


    Shutting down server on port 5000.



In [28]:
displacy.render(doc, style='dep', options=options, jupyter=True)

#### Visualize Named Entities

In [29]:
displacy.render(doc, style='ent', jupyter=True)

### Read BBC Data

In [32]:
files = Path('data', 'bbc').glob('**/*.txt')
bbc_articles = []
for i, file in enumerate(files):
    _, _, topic, file_name = file.parts
    with file.open(encoding='latin1') as f:
        lines = f.readlines()
        body = ' '.join([l.strip() for l in lines[1:]]).strip()
        bbc_articles.append(body)

In [33]:
len(bbc_articles)

2225

In [34]:
bbc_articles[0]

'Voting is under way for the annual Bloggies which recognise the best web blogs - online spaces where people publish their thoughts - of the year.  Nominations were announced on Sunday, but traffic to the official site was so heavy that the website was temporarily closed because of too many visitors. Weblogs have been nominated in 30 categories, from the top regional blog, to the best-kept-secret blog. Blogs had a huge year, with a top US dictionary naming "blog" word of 2004. Technorati, a blog search engine, tracks about six million blogs and says that more than 12,000 are added daily. A blog is created every 5.8 seconds, according to US research think-tank Pew Internet and American Life, but less than 40% of the total are updated at least once every two months.  Nikolai Nolan, who has run the Bloggies for the past five years, told the BBC News website he was not too surprised by the amount of voters who crowded the site. "The awards always get a lot of traffic; this was just my firs

### Parse first article through Pipeline

In [35]:
nlp.pipe_names

['tagger', 'parser', 'ner']

In [36]:
doc = nlp(bbc_articles[0])
type(doc)

spacy.tokens.doc.Doc

### Detect sentence boundary
Sentence boundaries are calculated from the syntactic parse tree, so features such as punctuation and capitalisation play an important but non-decisive role in determining the sentence boundaries. 

Usually this means that the sentence boundaries will at least coincide with clause boundaries, even given poorly punctuated text.

In [37]:
sentences = [s for s in doc.sents]
sentences[:3]

[Voting is under way for the annual Bloggies which recognise the best web blogs - online spaces where people publish their thoughts - of the year.  ,
 Nominations were announced on Sunday, but traffic to the official site was so heavy that the website was temporarily closed because of too many visitors.,
 Weblogs have been nominated in 30 categories, from the top regional blog, to the best-kept-secret blog.]

In [38]:
get_attributes(sentences[0])

['as_doc', 'doc', 'end', 'end_char', 'ent_id', 'ent_id_', 'ents', 'get_extension', 'get_lca_matrix', 'has_extension', 'has_vector', 'label', 'label_', 'lefts', 'lemma_', 'lower_', 'merge', 'n_lefts', 'n_rights', 'noun_chunks', 'orth_', 'remove_extension', 'rights', 'root', 'sent', 'sentiment', 'set_extension', 'similarity', 'start', 'start_char', 'string', 'subtree', 'text', 'text_with_ws', 'to_array', 'upper_', 'vector', 'vector_norm', 'vocab'] 

In [39]:
pd.DataFrame([[t.text, t.pos_, spacy.explain(t.pos_)] for t in sentences[0]], 
             columns=['Token', 'POS Tag', 'Meaning']).head(15)

,Token,POS Tag,Meaning
0,Voting,NOUN,noun
1,is,VERB,verb
2,under,ADP,adposition
3,way,NOUN,noun
4,for,ADP,adposition
5,the,DET,determiner
6,annual,ADJ,adjective
7,Bloggies,PROPN,proper noun
8,which,ADJ,adjective
9,recognise,VERB,verb


In [40]:
options = {'compact': True, 'bg': '#09a3d5',
           'color': 'white', 'font': 'Source Sans Pro'}
displacy.render(sentences[0].as_doc(), style='dep', jupyter=True, options=options)

In [41]:
for t in sentences[0]:
    if t.ent_type_:
        print('{} | {} | {}'.format(t.text, t.ent_type_, spacy.explain(t.ent_type_)))

annual | DATE | Absolute or relative dates or periods
the | DATE | Absolute or relative dates or periods
year | DATE | Absolute or relative dates or periods


In [42]:
displacy.render(sentences[0].as_doc(), style='ent', jupyter=True)

### Named Entity-Recognition with textacy

In [43]:
from textacy.extract import named_entities
entities = [e.text for e in named_entities(doc)]
pd.Series(entities).value_counts().head()

year                          4
US                            2
annual                        2
Tsunami Blog                  2
South-East Asia Earthquake    2
dtype: int64

### N-Grams with textacy

In [44]:
from textacy.extract import ngrams
pd.Series([n.text for n in ngrams(doc, n=2, min_freq=2)]).value_counts()

Tsunami Blog       2
annual Bloggies    2
East Asia          2
Asia Earthquake    2
dtype: int64

### The spaCy streaming Pipeline API

In [45]:
iter_texts = (bbc_articles[i] for i in range(len(bbc_articles)))
for i, doc in enumerate(nlp.pipe(iter_texts, batch_size=50, n_threads=8)):
    if i % 100 == 0:
        print(i, end = ' ')
    assert doc.is_parsed

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 

### Multi-language Features

#### Create a Spanish Language Object

In [46]:
model = {}
for language in ['en', 'es']:
    model[language] = spacy.load(language) 

#### Read bilingual TED2013 samples

In [48]:
text = {}
path = Path('data/TED')
for language in ['en', 'es']:
    file_name = path /  'TED2013_sample.{}'.format(language)
    text[language] = file_name.read_text()

#### Sentence Boundaries English vs Spanish

In [49]:
parsed, sentences = {}, {}
for language in ['en', 'es']:
    parsed[language] = model[language](text[language])
    sentences[language] = list(parsed[language].sents)
    print('Sentences:', language, len(sentences[language]))

Sentences: en 19
Sentences: es 22


In [50]:
for i, (en, es) in enumerate(zip(sentences['en'], sentences['es']), 1):
    print('\n', i)
    print('English:\t', en)
    print('Spanish:\t', es)
    if i > 5: 
        break


 1
English:	 There's a tight and surprising link between the ocean's health and ours, says marine biologist Stephen Palumbi.
Spanish:	 Existe una estrecha y sorprendente relación entre nuestra salud y la salud del océano, dice el biologo marino Stephen Palumbi.

 2
English:	 He shows how toxins at the bottom of the ocean food chain find their way into our bodies, with a shocking story of toxic contamination from a Japanese fish market.
Spanish:	 Nos muestra, através de una impactante historia acerca de la contaminación tóxica en el mercado pesquero japonés, como las toxinas de la cadena alimenticia del fondo oceánico llegan a nuestro cuerpo.

 3
English:	 His work points a way forward for saving the oceans' health -- and humanity's. fish,health,mission blue,oceans,science 899 Stephen Palumbi: Following the mercury trail It can be a very complicated thing, the ocean.
Spanish:	 fish,health,mission blue,oceans,science 899 Stephen Palumbi: Siguiendo el camino del mercurio.

 4
English:	 A

#### POS Tagging English vs Spanish

In [51]:
pos = {}
for language in ['en', 'es']:
    pos[language] = pd.DataFrame([[t.text, t.pos_, spacy.explain(t.pos_)] for t in sentences[language][0]],
                                 columns=['Token', 'POS Tag', 'Meaning'])

In [52]:
bilingual_parsed = pd.concat([pos['en'], pos['es']], axis=1)
bilingual_parsed.head(5).to_csv('data/bilingual.csv', index=False)
bilingual_parsed.head(15)

,Token,POS Tag,Meaning,Token,POS Tag,Meaning
0,There,ADV,adverb,Existe,VERB,verb
1,'s,VERB,verb,una,DET,determiner
2,a,DET,determiner,estrecha,ADJ,adjective
3,tight,ADJ,adjective,y,CONJ,conjunction
4,and,CCONJ,coordinating conjunction,sorprendente,ADJ,adjective
5,surprising,ADJ,adjective,relación,NOUN,noun
6,link,NOUN,noun,entre,ADP,adposition
7,between,ADP,adposition,nuestra,DET,determiner
8,the,DET,determiner,salud,NOUN,noun
9,ocean,NOUN,noun,y,CONJ,conjunction


In [53]:
displacy.render(sentences['es'][0].as_doc(), style='dep', jupyter=True, options=options)